# Intro Tutorial to Snorkel MeTaL

**[Currently under construction]**

In this tutorial...

Generate synthetic data

In [1]:
%%time
from metal.utils import split_data
from synthetic.generate import singletask_synthetic

n = 10000 # Number of data points
m = 10    # Number of labeling sources
k = 2     # Cardinality of task

D, L, X, Y, deps = singletask_synthetic(n, m, k)
Ds, Ls, Xs, Ys = split_data(D, L, X, Y, splits=[0.8, 0.1, 0.1])

CPU times: user 17.4 s, sys: 280 ms, total: 17.7 s
Wall time: 17.9 s


Train and score label model

In [4]:
%%time
from metal.label_model import LabelModel

label_model = LabelModel(k)
label_model.train(Ls[0], deps=deps)
score = label_model.score(Ls[2], Ys[2], verbose=False)
Y_p = label_model.predict(Ls[0])
print(f"Label Model Accuracy={score}")

Computing O...
Estimating \mu...
[Epoch 0] Loss: 13.137172
[Epoch 10] Loss: 3.693144
[Epoch 20] Loss: 1.007804
[Epoch 30] Loss: 0.046628
[Epoch 40] Loss: 0.091219
[Epoch 50] Loss: 0.060271
[Epoch 60] Loss: 0.014713
[Epoch 70] Loss: 0.002799
[Epoch 80] Loss: 0.002833
[Epoch 90] Loss: 0.001579
[Epoch 99] Loss: 0.000958
Label Model Accuracy=0.988
CPU times: user 722 ms, sys: 37.6 ms, total: 760 ms
Wall time: 281 ms


Train and score end model

In [5]:
%%time
from metal.end_model import EndModel

end_model = EndModel(layer_out_dims=[1000,10])
end_model.train(Xs[0], Y_p, Xs[1], Ys[1], n_epochs=5)
score = end_model.score(Xs[2], Ys[2], verbose=False)
print(f"End Model Accuracy: {score}")


Network architecture:
Sequential(
  (0): IdentityModule()
  (1): Sequential(
    (0): Linear(in_features=1000, out_features=10, bias=True)
    (1): ReLU()
  )
  (2): Linear(in_features=10, out_features=2, bias=True)
)



/Users/bradenjh/anaconda3/envs/metal/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Saving model at iteration 0 with best score 0.998
[E:0]	Train Loss: 0.113	Dev score: 0.998
[E:1]	Train Loss: 0.059	Dev score: 0.986
[E:2]	Train Loss: 0.037	Dev score: 0.987
[E:3]	Train Loss: 0.021	Dev score: 0.986
[E:4]	Train Loss: 0.015	Dev score: 0.988
Restoring best model from iteration 0 with score 0.998
Finished Training
Confusion Matrix (Dev)
        y=1    y=2   
 l=1    479     6    
 l=2     6     509   
End Model Accuracy: 0.978
CPU times: user 10.6 s, sys: 1.54 s, total: 12.1 s
Wall time: 2.01 s
